In [8]:
import os
import sys

# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath("./../src"))

import datetime
import importlib

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tqdm
from sklearnex import patch_sklearn

import chorus_machine_learning_helper
import data_loader
import denoising_autoencoder
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

importlib.reload(chorus_machine_learning_helper)
importlib.reload(denoising_autoencoder)
importlib.reload(data_loader)

import scipy.stats

from sklearnex import patch_sklearn
patch_sklearn()
import sklearn

%matplotlib qt

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [9]:
VERSION = "v1"
MODEL_TYPE = "LOWER_BAND"

pdata_folder = os.path.abspath("./../processed_data/chorus_neural_network/")
output_folder = os.path.join(pdata_folder, "models", VERSION)

T_SIZE = 31860 / 2

In [ ]:
POES_temp = data_loader.load_raw_data_from_config(
    id=["POES", "SEM", "MPE"],
    start=datetime.datetime(year=2000, month=1, day=1),
    end=datetime.datetime(year=2000, month=1, day=2),
    satellite="n15",
)

ENERGIES = POES_temp["energy"][0]
DIFF_E = ENERGIES[1:] - ENERGIES[:-1]

In [ ]:
time_chunked_global_maps = []

x_edges = None
y_edges = None

nx = 60
ny = 12

for year in range(1998, 2024):

    tstart = datetime.datetime(year=year, month=1, day=1, tzinfo=datetime.UTC).timestamp()
    tend = datetime.datetime(year=year + 1, month=1, day=1, tzinfo=datetime.UTC).timestamp()

    POES = chorus_machine_learning_helper.load_MPE_year(year=year)

    for tcurr in tqdm.tqdm(np.arange(tstart, tend, T_SIZE)):

        POES_L = []
        POES_MLT = []
        POES_FLUX = []

        for SAT in POES:
            TIME_RANGE = np.searchsorted(SAT["UNIX_TIME"], v=[tcurr, tcurr + T_SIZE])
            POES_L.append(SAT["L"][TIME_RANGE[0] : TIME_RANGE[-1]])
            POES_MLT.append(SAT["MLT"][TIME_RANGE[0] : TIME_RANGE[-1]])
            POES_FLUX.append(SAT["BLC_Flux"][TIME_RANGE[0] : TIME_RANGE[-1], :])


        POES_L = np.hstack(POES_L)
        POES_MLT = np.hstack(POES_MLT)
        POES_FLUX = np.vstack(POES_FLUX)

        if POES_L.shape[0] == 0:
            continue

        UNDER_130_KeV_FLUX = np.nansum((POES_FLUX[:, :7] * DIFF_E[:7]), axis=1)
        TOTAL_FLUX = np.nansum((POES_FLUX[:, :-1] * DIFF_E), axis=1)
        FLUX_RATIO = UNDER_130_KeV_FLUX / TOTAL_FLUX

        hist_count, _, _ = np.histogram2d(POES_L, POES_MLT, bins=[nx, ny], range=[[2, 8], [0, 24]])
        hist_sum, x_edges, y_edges = np.histogram2d(POES_L, POES_MLT, bins=[nx, ny], range=[[2, 8], [0, 24]], weights=TOTAL_FLUX)

        not_zero = (hist_sum != 0)

        hist_avg = np.full_like(hist_sum, fill_value=np.nan)
        hist_avg[not_zero] = hist_sum[not_zero] / hist_count[not_zero]

        time_chunked_global_maps.append(hist_avg)
        break
    break

In [ ]:
time_chunked_global_maps = np.vstack([np.expand_dims(time_chunked_global_maps[i], axis=0) for i in range(len(time_chunked_global_maps))])

print(time_chunked_global_maps.shape)

In [ ]:
np.savez(
    file=os.path.join(output_folder, rf"time_chunked_global_flux_maps_poes_{VERSION}_1998_2024.npz"),
    time_chunked_global_maps=time_chunked_global_maps,
)

In [10]:
data = np.load(
    file=os.path.join(output_folder, rf"time_chunked_global_flux_maps_poes_{VERSION}_1998_2024.npz"),
)

time_chunked_global_maps = data["time_chunked_global_maps"]

data.close()

In [11]:
%%time

fold_results, trained_models, transformers = denoising_autoencoder.cross_validate_model(
    time_chunked_global_maps,
    denoising_autoencoder.ConvDenoiser,
    denoising_autoencoder.Critic,
    n_splits=10,
    num_epochs=15,
    batch_size=16,
    denoiser_learning_rate=4e-5,
    critic_learning_rate=1e-6,
)


Fold 1/10

Creating Data Loaders
Creating Denoiser and Critic
Training...



100%|██████████████████████████████████████████████████████████████████████████████| 2629/2629 [01:16<00:00, 34.43it/s]


Testing Model on Test Data...

Epoch [1/15]
Averages:
Training Critic Loss: -53101.7926
Trainin Generator Loss: 189360.3922
Training MSE : 14147532.211815
Test Critic Loss: -4183.5464
Test Generator Loss: 30363.2285
Test MSE : 3999.504639



100%|██████████████████████████████████████████████████████████████████████████████| 2629/2629 [01:15<00:00, 34.87it/s]


Testing Model on Test Data...

Epoch [2/15]
Averages:
Training Critic Loss: -2607.5372
Trainin Generator Loss: 27714.4833
Training MSE : 1939.507362
Test Critic Loss: -1395.4346
Test Generator Loss: 11442.5488
Test MSE : 134.357315



...

100%|██████████████████████████████████████████████████████████████████████████████| 2629/2629 [01:17<00:00, 33.96it/s]


Testing Model on Test Data...

Epoch [14/15]
Averages:
Training Critic Loss: -2706.1829
Trainin Generator Loss: 2059.0172
Training MSE : 0.712913
Test Critic Loss: -1746.6808
Test Generator Loss: 1030.0348
Test MSE : 0.865555



100%|██████████████████████████████████████████████████████████████████████████████| 2629/2629 [01:16<00:00, 34.28it/s]


Testing Model on Test Data...

Epoch [15/15]
Averages:
Training Critic Loss: -1964.4491
Trainin Generator Loss: -733.6463
Training MSE : 0.725862
Test Critic Loss: -2229.4343
Test Generator Loss: 1004.9104
Test MSE : 0.831410

Testing Final Model on Training Data...

Testing Final Model on Test Data...

Results:

Train MSE : 0.8311309218406677
Train g_loss : 1020.1344604492188
Train c_loss : -2258.065185546875
Test MSE : 0.8334143757820129
Test g_loss : 997.2138061523438
 Test c_loss : -2226.29931640625



TypeError: mean(): argument 'input' (position 1) must be Tensor, not list

In [ ]:
first_example = torch.from_numpy(np.expand_dims(next(iter(train_loader))[0][0], axis=0)).type(torch.float32)
was_nan = torch.isnan(first_example)

nan_removed = np.nansum([was_nan.cpu() * transformed_averages[np.newaxis, np.newaxis, :, :], (~was_nan.cpu()) * first_example], axis=0)
targets = torch.from_numpy(nan_removed).to(device).type(torch.float32)
model.eval()
predictions = model(targets, (~was_nan).type(torch.float32).to(device)).detach().cpu().numpy()
predictions = PT.inverse_transform(predictions.reshape(targets.shape[2] * targets.shape[3], 1)).reshape(targets.shape[2], targets.shape[3])
targets = targets.detach().cpu()

In [ ]:
# Create mesh for plotting
_X, _Y = np.meshgrid(x_edges, y_edges)
print(targets.shape)
targets[was_nan] = np.nan
first_example_flux = PT.inverse_transform(targets[0][0].reshape(targets.shape[2] * targets.shape[3], 1)).reshape(targets.shape[2], targets.shape[3])
predictions = predictions

plt.figure(figsize=(10, 8))
plt.xlim((x_edges[0], x_edges[-1]))
plt.ylim((y_edges[0], y_edges[-1]))
plt.pcolormesh(_X,
               _Y,
               np.abs(first_example_flux.T),
               cmap='viridis',
               norm=matplotlib.colors.SymLogNorm(linthresh = 1, vmin = 0, vmax=1e8))
cbar = plt.colorbar()
cbar.set_label("Average Flux", loc="center", labelpad=25, rotation=270)
plt.xlabel("MLT")
plt.ylabel("L")
plt.title("Flux vs L vs MLT")
plt.show()

In [ ]:
# Create mesh for plotting
_X, _Y = np.meshgrid(x_edges, y_edges)


plt.figure(figsize=(10, 8))
plt.xlim((x_edges[0], x_edges[-1]))
plt.ylim((y_edges[0], y_edges[-1]))
plt.pcolormesh(_X,
               _Y,
               np.abs(predictions.T),
               cmap='viridis',
               norm=matplotlib.colors.SymLogNorm(linthresh = 1, vmin = 0, vmax=1e8))
cbar = plt.colorbar()
cbar.set_label("Average Flux", loc="center", labelpad=25, rotation=270)
plt.xlabel("L")
plt.ylabel("MLT")
plt.title("Flux vs L vs MLT")
plt.show()